In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('~/Downloads/fhv_tripdata_2021-01.parquet')
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


**Q1: Read the data for January. How many records are there?**
A: 1154112

In [3]:
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
df['duration'].mean()

Timedelta('0 days 00:19:10.033445627')

**Q2: What's the average trip duration in January?**
A: 19m 10s

In [4]:
df_clean = df[df['duration'].between(pd.Timedelta(1, "m"), pd.Timedelta(60, "m"))]
df_clean

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,0 days 00:17:00
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,0 days 00:17:00
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,0 days 00:08:17
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,0 days 00:15:13
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,0 days 00:09:03
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,0 days 00:08:45
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,0 days 00:57:36
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,0 days 00:16:12
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,0 days 00:19:26


In [5]:
df_clean['PUlocationID'] = df_clean['PUlocationID'].fillna(-1)
df_clean['DOlocationID'] = df_clean['DOlocationID'].fillna(-1)
df_clean['PUlocationID'].value_counts()[-1.0] / len(df_clean)

<ipython-input-5-dbaefffd25a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['PUlocationID'] = df_clean['PUlocationID'].fillna(-1)
<ipython-input-5-dbaefffd25a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['DOlocationID'] = df_clean['DOlocationID'].fillna(-1)


0.8352732770722617

**Q3: What's the fractions of missing values for the pickup location ID?**
A: 83%

In [6]:
categorical = ['PUlocationID', 'DOlocationID']
df_clean[categorical] = df_clean[categorical].astype(str)

train_dicts = df_clean[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_1 = dv.fit_transform(train_dicts)
X_1.shape


<ipython-input-6-f335cf651c6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical] = df_clean[categorical].astype(str)


(1109826, 525)

**Q4: What's the dimensionality of this matrix? (The number of columns).**
A: 525

In [7]:
df_clean['duration_min'] = df_clean.duration.apply(lambda td: td.total_seconds() / 60)


<ipython-input-7-f987a0755b14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_min'] = df_clean.duration.apply(lambda td: td.total_seconds() / 60)


In [8]:
y_train = df_clean['duration_min']
lr = LinearRegression()
lr.fit(X_1, y_train)

LinearRegression()

In [9]:
y_pred = lr.predict(X_1)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(rmse)

10.52851938897048


**Q5: RMSE on train**
A: 10.5%

In [10]:
df_validate = pd.read_parquet('~/Downloads/fhv_tripdata_2021-02.parquet')
df_validate['PUlocationID'].fillna(-1)
df_validate['DOlocationID'].fillna(-1)
df_validate['duration'] = df_validate['dropOff_datetime'] - df_validate['pickup_datetime']
df_validate = df_validate[df_validate['duration'].between(pd.Timedelta(1, "m"), pd.Timedelta(60, "m"))]
df_validate['duration_min'] = df_validate.duration.apply(lambda td: td.total_seconds() / 60)

df_validate[categorical] = df_validate[categorical].astype(str)

train_dicts = df_validate[categorical].to_dict(orient='records')

X_val = dv.transform(train_dicts)
y_pred = lr.predict(X_val)


In [11]:
y_val = df_validate['duration_min']
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(rmse)

11.36412428666764


**Q6: RMSE on validation?**
A: 11.4%